In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

#we actually need to concatenate a bunch of files to get who annotated what  
stem = "/home/blitt/projects/podcasts/annotation/hostGuestAnnotation/annotation_output/"
folderNames  = ["5a4d8a8f0b817800013032ea", "5ece9e3b3a19db0cf5af6692", "6022ad31accb29025f079d1c", "65537e115e58fe12b3fcf2f2", "6615cd319d6d5ee3879758ee", "5e318768c7b20d000d0830ce", "5ff66092dfb1e6211decef3e", "6440d887ac943c907678ec9f", "65e739f565a8c818b009f83b"]
dfList = []

for fName in folderNames: 
    fPath = stem + fName + "/annotated_instances.jsonl" 
    currDf = pd.read_json(fPath, orient="records", lines=True)
    currDf["annotatorId"] = fName
    dfList.append(currDf)
df = pd.concat(dfList)

dictList = list(df["label_annotations"])
labels = [int(list(item["sentiment"].values())[0]) for item in dictList]
df["annotation"] = labels

In [18]:
df.head() 

,id,displayed_text,label_annotations,span_annotations,behavioral_data,annotatorId,annotation
0,https://feeds.soundcloud.com/stream/825188326-...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'neither': '3'}},{},{'time_string': 'Time spent: 0d 0h 0m 12s '},5a4d8a8f0b817800013032ea,3
1,https://www.buzzsprout.com/998860/3752489-corn...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'host': '1'}},{},{'time_string': 'Time spent: 0d 0h 0m 18s '},5a4d8a8f0b817800013032ea,1
2,https://anchor.fm/s/11028184/podcast/play/1421...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'guest': '2'}},{},{'time_string': 'Time spent: 0d 0h 0m 17s '},5a4d8a8f0b817800013032ea,2
3,https://api.spreaker.com/download/episode/2686...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'neither': '3'}},{},{'time_string': 'Time spent: 0d 0h 0m 16s '},5a4d8a8f0b817800013032ea,3
4,https://feeds.soundcloud.com/stream/829978630-...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'guest': '2'}},{},{'time_string': 'Time spent: 0d 0h 0m 12s '},5a4d8a8f0b817800013032ea,2


In [19]:
#find high disagreement examples
instGrouped = df[["id", "annotatorId", "annotation", "displayed_text"]].groupby("id").agg(list)
instGrouped["displayed_text"] = instGrouped["displayed_text"].apply(lambda x: x[0])

In [20]:

def getNonModal(inRow): 
    counts = Counter(inRow["annotation"])
    modalVal, modeCount = counts.most_common(1)[0]

    return [modalVal, modeCount]

instGrouped[["modalVal", "modalCount"]] = instGrouped.apply(getNonModal, axis=1).to_list()

In [21]:
def pPrint(inText): 
    tList = inText.split()
    lineLen = 20
    currText = ""
    for count, item in enumerate(tList): 
        if count % lineLen == 0: 
            print(currText)
            currText = ""
        currText += item + " "

In [23]:
#totalDisDf = df.set_index("id", drop=False).loc[totalDisList].drop_duplicates("id") 
totalDisDf = instGrouped[instGrouped["modalCount"] == 3]
for i, row in totalDisDf.iterrows(): 
    print(i)
    print(row["annotation"])
    pPrint(row["displayed_text"])

http://dts.podtrac.com/redirect.mp3/feeds.soundcloud.com/stream/834697933-siggraph-spotlight-35-real-time-ray-tracing-and-realism.mp3
[1, 1, 1]

<span style='background-color:#00FF00'>Target Entity: Mark Alano</span><br><br><strong>Podcast Description:</strong><br>SIGGRAPH Spotlight is a podcast that explores cutting-edge projects and topics in computer graphics and 
interactive techniques, one episode at a time. Episodes are hosted by ACM SIGGRAPH leaders and feature experts spanning everything from 
animation and visual effects to engineering and virtual reality. This show was founded as an extension of ACM SIGGRAPH's annual 
conference.<br><br><strong>Podcast Episode Description:</strong><br>ABOUT THE EPISODEIn this episode of SIGGRAPH Spotlight, SIGGRAPH 2020 Real-Time Live! Chair Marc Olano sits down with 
four incredible creators who are pushing the boundaries of innovation in real-time graphics. Arisa Scott of Unity Technologies, Eric Haines 
of NVIDIA, Mike Hardison of Blizzard

In [ ]:
#host: 
"https://anchor.fm/s/7f12654/podcast/play/14320049/https%3A%2F%2Fd3ctxlq1ktw2nl.cloudfront.net%2Fstaging%2F2020-4-26%2F76850511-44100-2-e778a17d843dd.m4a"

#guest: 
"https://anchor.fm/s/11028184/podcast/play/14212379/https%3A%2F%2Fd3ctxlq1ktw2nl.cloudfront.net%2Fproduction%2F2020-4-24%2F76170728-44100-2-109d447b04488.mp3"

#neither: 
"https://anchor.fm/s/24044d44/podcast/play/14503148/https%3A%2F%2Fd3ctxlq1ktw2nl.cloudfront.net%2Fstaging%2F2020-4-30%2F78020478-44100-1-1e08e64348a62916.mp3"